**Name:** Rakesh Kumar K S

**Reg. no:** 20BAI1055

**DL Lab 4:** CNN Hyperparameter tuning - Manual and automated

Unzipping the dataset of images

In [6]:
!ls

archive.zip  sample_data


In [7]:
!unzip archive.zip

Archive:  archive.zip
  inflating: SheepFaceImages/Marino/00.jpgMA8.jpg  
  inflating: SheepFaceImages/Marino/000037MA2.jpg  
  inflating: SheepFaceImages/Marino/000041MD4.jpg  
  inflating: SheepFaceImages/Marino/000044MA2.jpg  
  inflating: SheepFaceImages/Marino/000046MD4.jpg  
  inflating: SheepFaceImages/Marino/000086MD4.jpg  
  inflating: SheepFaceImages/Marino/000091MD4.jpg  
  inflating: SheepFaceImages/Marino/000105MA6.jpg  
  inflating: SheepFaceImages/Marino/000142MD4.jpg  
  inflating: SheepFaceImages/Marino/000147MD4.jpg  
  inflating: SheepFaceImages/Marino/000152MD4.jpg  
  inflating: SheepFaceImages/Marino/000198MD4.jpg  
  inflating: SheepFaceImages/Marino/000203MD4.jpg  
  inflating: SheepFaceImages/Marino/000206MA9.jpg  
  inflating: SheepFaceImages/Marino/000208MD4.jpg  
  inflating: SheepFaceImages/Marino/000211MA9.jpg  
  inflating: SheepFaceImages/Marino/000217MA9.jpg  
  inflating: SheepFaceImages/Marino/000221MA9.jpg  
  inflating: SheepFaceImages/Marino/000224

Pre-processing the dataset, converting it to pixel values and saving it as a csv to import it later

In [105]:
import os

import numpy as np
import pandas as pd

from skimage import io
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import morphology

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [21]:
sheep_data = {}
image_filename_index = 0

In [22]:
for sheep_breed in os.listdir("./SheepFaceImages"):
    print(sheep_breed)
    for image_filename in os.listdir("./SheepFaceImages/" + sheep_breed):
        sheep = io.imread("./SheepFaceImages/" + sheep_breed + "/" + image_filename)
        sheep = rgb2gray(sheep)
        sheep = resize(sheep, (90,78), anti_aliasing=False)
        sheep = sheep.reshape(sheep.shape[0] * sheep.shape[1])
        sheep = np.append(sheep, str(sheep_breed))
        sheep_data[image_filename_index] = sheep
        image_filename_index = image_filename_index + 1

Poll Dorset
Suffolk
White Suffolk
Marino


In [25]:
column_names = [str(i) for i in range(0,90*78)]
column_names = np.append(column_names, 'breed')
df = pd.DataFrame.from_dict(sheep_data, orient='index', columns=column_names)
df.to_csv('Sheep_breed_dataset.csv', index=None)

Importing the generated CSV dataset

In [27]:
df = pd.read_csv('Sheep_breed_dataset.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,7011,7012,7013,7014,7015,7016,7017,7018,7019,breed
0,0.408562,0.384150,0.326198,0.285959,0.480415,0.526635,0.499246,0.518941,0.518615,0.531175,...,0.288533,0.296376,0.308141,0.320166,0.349600,0.371169,0.372138,0.393576,0.380885,Poll Dorset
1,0.677487,0.639851,0.781991,0.634100,0.583122,0.774073,0.802913,0.849024,0.781168,0.789087,...,0.561139,0.589546,0.639557,0.628740,0.633544,0.621964,0.598576,0.577676,0.609288,Poll Dorset
2,0.189061,0.230135,0.299128,0.294919,0.507569,0.645683,0.672919,0.639631,0.550700,0.466744,...,0.128568,0.127615,0.128584,0.136938,0.130086,0.127134,0.130086,0.125173,0.117330,Poll Dorset
3,0.665366,0.708786,0.665329,0.671811,0.639011,0.632008,0.716082,0.675276,0.666681,0.770603,...,0.392001,0.468557,0.494285,0.479729,0.453259,0.438510,0.394348,0.332870,0.194321,Poll Dorset
4,0.827807,0.828321,0.788023,0.700601,0.379579,0.313108,0.340381,0.307121,0.259451,0.202675,...,0.373964,0.385772,0.399541,0.424689,0.462136,0.697925,0.751208,0.750039,0.680228,Poll Dorset


Splitting the data set into input and output

In [32]:
X = df.drop(['breed'], axis=1).values

encoder = LabelEncoder()
df['breed'] = encoder.fit_transform(df['breed'])
Y = to_categorical(df['breed'])

In [34]:
X

array([[0.40856209, 0.38415033, 0.3261985 , ..., 0.37213813, 0.39357603,
        0.3808854 ],
       [0.67748684, 0.63985068, 0.78199144, ..., 0.59857603, 0.5776759 ,
        0.6092881 ],
       [0.18906089, 0.23013502, 0.29912769, ..., 0.13008619, 0.12517333,
        0.1173302 ],
       ...,
       [0.45981434, 0.51077294, 0.38682959, ..., 0.03332196, 0.03332196,
        0.03332196],
       [0.42497002, 0.50280667, 0.68773288, ..., 0.71499264, 0.71197521,
        0.72272684],
       [0.63287471, 0.69416011, 0.70493353, ..., 0.1735351 , 0.19229895,
        0.09795503]])

In [35]:
Y

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

Splitting the dataset into train, test and validation set

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3, shuffle=True, stratify=Y, random_state=0)

In [41]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.3, shuffle=True, stratify=y_train, random_state=0)

In [47]:
X_train = X_train.reshape(len(X_train),90,78,1)
X_val = X_val.reshape(len(X_val),90,78,1)
X_test = X_test.reshape(len(X_test),90,78,1)

Creating and training a base model

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout

In [56]:
base_model = Sequential()

base_model.add(Conv2D(filters=64, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
base_model.add(Flatten())
base_model.add(Dense(64))
base_model.add(Dense(32))
base_model.add(Dense(4, activation='softmax'))

# compile the model
base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [57]:
base_model.fit(X_train, y_train, batch_size=128, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
7/7 [==============================] - 1s 61ms/step - loss: 38.4359 - accuracy: 0.2880 - val_loss: 51.1884 - val_accuracy: 0.2635
Epoch 2/5
7/7 [==============================] - 0s 35ms/step - loss: 30.9454 - accuracy: 0.3439 - val_loss: 8.7126 - val_accuracy: 0.4221
Epoch 3/5
7/7 [==============================] - 0s 30ms/step - loss: 6.8089 - accuracy: 0.3390 - val_loss: 5.8428 - val_accuracy: 0.3428
Epoch 4/5
7/7 [==============================] - 0s 30ms/step - loss: 2.8838 - accuracy: 0.4022 - val_loss: 1.5983 - val_accuracy: 0.4618
Epoch 5/5
7/7 [==============================] - 0s 29ms/step - loss: 1.6048 - accuracy: 0.4994 - val_loss: 1.5621 - val_accuracy: 0.4901


We got bad accuracy on validation set. Let us try hyper parameter tuning manually

Adding activation functions to the model

In [58]:
model1 = Sequential()

model1.add(Conv2D(filters=64, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
model1.add(Flatten())
model1.add(Dense(64, activation='relu'))
model1.add(Dense(32, activation='relu'))
model1.add(Dense(4, activation='softmax'))

# compile the model
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [59]:
model1.fit(X_train, y_train, batch_size=128, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
7/7 [==============================] - 1s 66ms/step - loss: 3.1501 - accuracy: 0.3098 - val_loss: 1.4574 - val_accuracy: 0.2521
Epoch 2/5
7/7 [==============================] - 0s 30ms/step - loss: 1.6912 - accuracy: 0.2649 - val_loss: 1.2817 - val_accuracy: 0.2663
Epoch 3/5
7/7 [==============================] - 0s 34ms/step - loss: 1.3366 - accuracy: 0.3803 - val_loss: 1.1490 - val_accuracy: 0.5071
Epoch 4/5
7/7 [==============================] - 0s 35ms/step - loss: 1.1470 - accuracy: 0.4812 - val_loss: 1.1107 - val_accuracy: 0.4703
Epoch 5/5
7/7 [==============================] - 0s 29ms/step - loss: 1.0255 - accuracy: 0.6136 - val_loss: 0.9269 - val_accuracy: 0.6544


We can notice that our validation accuracy has increased quite significantly now compared to before

Adjusting learning rate

In [62]:
model2 = Sequential()

model2.add(Conv2D(filters=64, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
model2.add(Flatten())
model2.add(Dense(64, activation='relu'))
model2.add(Dense(32, activation='relu'))
model2.add(Dense(4, activation='softmax'))

# compile the model
model2.compile(optimizer=tf.keras.optimizers.Adam(0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [63]:
model2.fit(X_train, y_train, batch_size=128, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
7/7 [==============================] - 1s 64ms/step - loss: 1.3723 - accuracy: 0.2564 - val_loss: 1.3447 - val_accuracy: 0.3598
Epoch 2/5
7/7 [==============================] - 0s 33ms/step - loss: 1.3340 - accuracy: 0.4058 - val_loss: 1.3103 - val_accuracy: 0.4731
Epoch 3/5
7/7 [==============================] - 0s 29ms/step - loss: 1.3010 - accuracy: 0.4459 - val_loss: 1.2722 - val_accuracy: 0.5439
Epoch 4/5
7/7 [==============================] - 0s 30ms/step - loss: 1.2660 - accuracy: 0.5164 - val_loss: 1.2426 - val_accuracy: 0.5581
Epoch 5/5
7/7 [==============================] - 0s 28ms/step - loss: 1.2368 - accuracy: 0.4957 - val_loss: 1.2127 - val_accuracy: 0.5581


The model isn't learning much in the last 3 epochs. Let us try increasing the no. of epochs as well

In [65]:
model2.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
7/7 [==============================] - 0s 49ms/step - loss: 0.9312 - accuracy: 0.6865 - val_loss: 0.9302 - val_accuracy: 0.6629
Epoch 2/50
7/7 [==============================] - 0s 30ms/step - loss: 0.9218 - accuracy: 0.7278 - val_loss: 0.9208 - val_accuracy: 0.6884
Epoch 3/50
7/7 [==============================] - 0s 29ms/step - loss: 0.9069 - accuracy: 0.7047 - val_loss: 0.9098 - val_accuracy: 0.6516
Epoch 4/50
7/7 [==============================] - 0s 35ms/step - loss: 0.8946 - accuracy: 0.6962 - val_loss: 0.9002 - val_accuracy: 0.6544
Epoch 5/50
7/7 [==============================] - 0s 29ms/step - loss: 0.8849 - accuracy: 0.7072 - val_loss: 0.8913 - val_accuracy: 0.6941
Epoch 6/50
7/7 [==============================] - 0s 29ms/step - loss: 0.8782 - accuracy: 0.7303 - val_loss: 0.8814 - val_accuracy: 0.6997
Epoch 7/50
7/7 [==============================] - 0s 30ms/step - loss: 0.8671 - accuracy: 0.6987 - val_loss: 0.8728 - val_accuracy: 0.6601
Epoch 8/50
7/7 [===========

We can also try adding Dropout layers to the above

In [66]:
model3 = Sequential()

model3.add(Conv2D(filters=64, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
model3.add(Flatten())
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.2, input_shape=(64,)))
model3.add(Dense(32, activation='relu'))
model3.add(Dense(4, activation='softmax'))

# compile the model
model3.compile(optimizer=tf.keras.optimizers.Adam(0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [67]:
model3.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
7/7 [==============================] - 1s 64ms/step - loss: 1.3826 - accuracy: 0.3123 - val_loss: 1.3661 - val_accuracy: 0.2691
Epoch 2/50
7/7 [==============================] - 0s 30ms/step - loss: 1.3628 - accuracy: 0.3548 - val_loss: 1.3457 - val_accuracy: 0.4249
Epoch 3/50
7/7 [==============================] - 0s 30ms/step - loss: 1.3469 - accuracy: 0.4216 - val_loss: 1.3200 - val_accuracy: 0.4844
Epoch 4/50
7/7 [==============================] - 0s 29ms/step - loss: 1.3268 - accuracy: 0.4459 - val_loss: 1.2972 - val_accuracy: 0.5382
Epoch 5/50
7/7 [==============================] - 0s 29ms/step - loss: 1.3097 - accuracy: 0.4714 - val_loss: 1.2776 - val_accuracy: 0.5694
Epoch 6/50
7/7 [==============================] - 0s 35ms/step - loss: 1.2918 - accuracy: 0.4921 - val_loss: 1.2596 - val_accuracy: 0.5552
Epoch 7/50
7/7 [==============================] - 0s 29ms/step - loss: 1.2720 - accuracy: 0.5115 - val_loss: 1.2430 - val_accuracy: 0.5666
Epoch 8/50
7/7 [===========

We can see that adding a Dropout layer didn't really improve the model. So we need not add it

Let us try playing around batch_size

In [84]:
model4 = Sequential()

model4.add(Conv2D(filters=64, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
model4.add(Flatten())
model4.add(Dense(64, activation='relu'))
model4.add(Dense(32, activation='relu'))
model4.add(Dense(4, activation='softmax'))

# compile the model
model4.compile(optimizer=tf.keras.optimizers.Adam(0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [85]:
model4.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
13/13 [==============================] - 1s 31ms/step - loss: 1.3710 - accuracy: 0.2527 - val_loss: 1.3331 - val_accuracy: 0.3853
Epoch 2/50
13/13 [==============================] - 0s 18ms/step - loss: 1.3187 - accuracy: 0.3706 - val_loss: 1.2858 - val_accuracy: 0.4164
Epoch 3/50
13/13 [==============================] - 0s 16ms/step - loss: 1.2732 - accuracy: 0.4289 - val_loss: 1.2473 - val_accuracy: 0.5014
Epoch 4/50
13/13 [==============================] - 0s 16ms/step - loss: 1.2318 - accuracy: 0.4872 - val_loss: 1.2068 - val_accuracy: 0.4986
Epoch 5/50
13/13 [==============================] - 0s 19ms/step - loss: 1.1940 - accuracy: 0.5079 - val_loss: 1.1767 - val_accuracy: 0.5354
Epoch 6/50
13/13 [==============================] - 0s 23ms/step - loss: 1.1600 - accuracy: 0.5443 - val_loss: 1.1503 - val_accuracy: 0.5212
Epoch 7/50
13/13 [==============================] - 0s 20ms/step - loss: 1.1322 - accuracy: 0.5431 - val_loss: 1.1214 - val_accuracy: 0.6034
Epoch 8/50
13

In [86]:
model4 = Sequential()

model4.add(Conv2D(filters=64, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
model4.add(Flatten())
model4.add(Dense(64, activation='relu'))
model4.add(Dense(32, activation='relu'))
model4.add(Dense(4, activation='softmax'))

# compile the model
model4.compile(optimizer=tf.keras.optimizers.Adam(0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [87]:
model4.fit(X_train, y_train, batch_size=256, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
4/4 [==============================] - 1s 97ms/step - loss: 1.3804 - accuracy: 0.2539 - val_loss: 1.3668 - val_accuracy: 0.3994
Epoch 2/50
4/4 [==============================] - 0s 48ms/step - loss: 1.3647 - accuracy: 0.4046 - val_loss: 1.3512 - val_accuracy: 0.4448
Epoch 3/50
4/4 [==============================] - 0s 42ms/step - loss: 1.3484 - accuracy: 0.4289 - val_loss: 1.3361 - val_accuracy: 0.4476
Epoch 4/50
4/4 [==============================] - 0s 44ms/step - loss: 1.3337 - accuracy: 0.4459 - val_loss: 1.3219 - val_accuracy: 0.4731
Epoch 5/50
4/4 [==============================] - 0s 42ms/step - loss: 1.3191 - accuracy: 0.4860 - val_loss: 1.3085 - val_accuracy: 0.4929
Epoch 6/50
4/4 [==============================] - 0s 46ms/step - loss: 1.3076 - accuracy: 0.5431 - val_loss: 1.2959 - val_accuracy: 0.5637
Epoch 7/50
4/4 [==============================] - 0s 42ms/step - loss: 1.2928 - accuracy: 0.5383 - val_loss: 1.2822 - val_accuracy: 0.4929
Epoch 8/50
4/4 [===========

There are no significant changes when we increase or decrease batch_size

Let us try changing the filter size

In [88]:
model5 = Sequential()

model5.add(Conv2D(filters=32, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
model5.add(Flatten())
model5.add(Dense(64, activation='relu'))
model5.add(Dense(32, activation='relu'))
model5.add(Dense(4, activation='softmax'))

# compile the model
model5.compile(optimizer=tf.keras.optimizers.Adam(0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [89]:
model5.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
7/7 [==============================] - 1s 56ms/step - loss: 1.3749 - accuracy: 0.2564 - val_loss: 1.3543 - val_accuracy: 0.3003
Epoch 2/50
7/7 [==============================] - 0s 27ms/step - loss: 1.3543 - accuracy: 0.2892 - val_loss: 1.3359 - val_accuracy: 0.3031
Epoch 3/50
7/7 [==============================] - 0s 24ms/step - loss: 1.3344 - accuracy: 0.3062 - val_loss: 1.3191 - val_accuracy: 0.2975
Epoch 4/50
7/7 [==============================] - 0s 22ms/step - loss: 1.3180 - accuracy: 0.3572 - val_loss: 1.3028 - val_accuracy: 0.3456
Epoch 5/50
7/7 [==============================] - 0s 21ms/step - loss: 1.3041 - accuracy: 0.3597 - val_loss: 1.2882 - val_accuracy: 0.3711
Epoch 6/50
7/7 [==============================] - 0s 20ms/step - loss: 1.2885 - accuracy: 0.3609 - val_loss: 1.2733 - val_accuracy: 0.3909
Epoch 7/50
7/7 [==============================] - 0s 21ms/step - loss: 1.2736 - accuracy: 0.4386 - val_loss: 1.2612 - val_accuracy: 0.4759
Epoch 8/50
7/7 [===========

In [90]:
model6 = Sequential()

model6.add(Conv2D(filters=128, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
model6.add(Flatten())
model6.add(Dense(64, activation='relu'))
model6.add(Dense(32, activation='relu'))
model6.add(Dense(4, activation='softmax'))

# compile the model
model6.compile(optimizer=tf.keras.optimizers.Adam(0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [91]:
model6.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
7/7 [==============================] - 1s 77ms/step - loss: 1.3641 - accuracy: 0.2430 - val_loss: 1.3304 - val_accuracy: 0.2720
Epoch 2/50
7/7 [==============================] - 0s 47ms/step - loss: 1.3225 - accuracy: 0.3086 - val_loss: 1.2845 - val_accuracy: 0.4419
Epoch 3/50
7/7 [==============================] - 0s 47ms/step - loss: 1.2751 - accuracy: 0.4350 - val_loss: 1.2484 - val_accuracy: 0.4986
Epoch 4/50
7/7 [==============================] - 0s 47ms/step - loss: 1.2392 - accuracy: 0.4739 - val_loss: 1.2178 - val_accuracy: 0.5666
Epoch 5/50
7/7 [==============================] - 0s 49ms/step - loss: 1.2088 - accuracy: 0.5504 - val_loss: 1.1894 - val_accuracy: 0.5156
Epoch 6/50
7/7 [==============================] - 0s 48ms/step - loss: 1.1847 - accuracy: 0.4824 - val_loss: 1.1667 - val_accuracy: 0.5892
Epoch 7/50
7/7 [==============================] - 0s 50ms/step - loss: 1.1590 - accuracy: 0.5699 - val_loss: 1.1438 - val_accuracy: 0.6062
Epoch 8/50
7/7 [===========

Having a smaller sized kernel seems like a better option here. Smaller kernels extract smaller details better

Finally, let us try different optimizers. We've aready tried Adam above, so let us try the others

In [94]:
model7 = Sequential()

model7.add(Conv2D(filters=32, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
model7.add(Flatten())
model7.add(Dense(64, activation='relu'))
model7.add(Dense(32, activation='relu'))
model7.add(Dense(4, activation='softmax'))

model7.compile(optimizer=tf.keras.optimizers.Adagrad(0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model7.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
7/7 [==============================] - 1s 57ms/step - loss: 1.3850 - accuracy: 0.2539 - val_loss: 1.3858 - val_accuracy: 0.2663
Epoch 2/50
7/7 [==============================] - 0s 27ms/step - loss: 1.3842 - accuracy: 0.2600 - val_loss: 1.3849 - val_accuracy: 0.2720
Epoch 3/50
7/7 [==============================] - 0s 25ms/step - loss: 1.3833 - accuracy: 0.2710 - val_loss: 1.3839 - val_accuracy: 0.2691
Epoch 4/50
7/7 [==============================] - 0s 22ms/step - loss: 1.3825 - accuracy: 0.2600 - val_loss: 1.3829 - val_accuracy: 0.2748
Epoch 5/50
7/7 [==============================] - 0s 21ms/step - loss: 1.3816 - accuracy: 0.2649 - val_loss: 1.3820 - val_accuracy: 0.2776
Epoch 6/50
7/7 [==============================] - 0s 21ms/step - loss: 1.3807 - accuracy: 0.2673 - val_loss: 1.3810 - val_accuracy: 0.2748
Epoch 7/50
7/7 [==============================] - 0s 20ms/step - loss: 1.3798 - accuracy: 0.2649 - val_loss: 1.3800 - val_accuracy: 0.2748
Epoch 8/50
7/7 [===========

Adagrad turned out to be really bad for the dataset. Let us try RMSprop

In [95]:
model8 = Sequential()

model8.add(Conv2D(filters=32, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
model8.add(Flatten())
model8.add(Dense(64, activation='relu'))
model8.add(Dense(32, activation='relu'))
model8.add(Dense(4, activation='softmax'))

model8.compile(optimizer=tf.keras.optimizers.RMSprop(0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model8.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
7/7 [==============================] - 1s 58ms/step - loss: 1.3728 - accuracy: 0.2539 - val_loss: 1.3466 - val_accuracy: 0.2436
Epoch 2/50
7/7 [==============================] - 0s 30ms/step - loss: 1.3423 - accuracy: 0.2722 - val_loss: 1.2987 - val_accuracy: 0.4136
Epoch 3/50
7/7 [==============================] - 0s 25ms/step - loss: 1.3057 - accuracy: 0.3913 - val_loss: 1.2731 - val_accuracy: 0.4193
Epoch 4/50
7/7 [==============================] - 0s 23ms/step - loss: 1.2678 - accuracy: 0.4411 - val_loss: 1.2475 - val_accuracy: 0.5864
Epoch 5/50
7/7 [==============================] - 0s 23ms/step - loss: 1.2406 - accuracy: 0.5334 - val_loss: 1.2213 - val_accuracy: 0.4419
Epoch 6/50
7/7 [==============================] - 0s 22ms/step - loss: 1.2201 - accuracy: 0.5188 - val_loss: 1.2141 - val_accuracy: 0.5127
Epoch 7/50
7/7 [==============================] - 0s 23ms/step - loss: 1.1935 - accuracy: 0.5358 - val_loss: 1.1774 - val_accuracy: 0.4844
Epoch 8/50
7/7 [===========

From the above manual tuning, we can conclude that we got best accuracy by using a kernel size of 32, Adam optimizer, 50 epochs, a learning rate of 0.00001, using reLu activation function, no dropout layer and 128 batch size. Let us create the final model and train it

In [96]:
model9 = Sequential()

model9.add(Conv2D(filters=32, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
model9.add(Flatten())
model9.add(Dense(64, activation='relu'))
model9.add(Dense(32, activation='relu'))
model9.add(Dense(4, activation='softmax'))

model9.compile(optimizer=tf.keras.optimizers.Adam(0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model9.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
7/7 [==============================] - 1s 58ms/step - loss: 1.3691 - accuracy: 0.2710 - val_loss: 1.3429 - val_accuracy: 0.2521
Epoch 2/50
7/7 [==============================] - 0s 24ms/step - loss: 1.3408 - accuracy: 0.2515 - val_loss: 1.3139 - val_accuracy: 0.2975
Epoch 3/50
7/7 [==============================] - 0s 22ms/step - loss: 1.3128 - accuracy: 0.3609 - val_loss: 1.2864 - val_accuracy: 0.4136
Epoch 4/50
7/7 [==============================] - 0s 21ms/step - loss: 1.2861 - accuracy: 0.3755 - val_loss: 1.2600 - val_accuracy: 0.3853
Epoch 5/50
7/7 [==============================] - 0s 21ms/step - loss: 1.2585 - accuracy: 0.3937 - val_loss: 1.2376 - val_accuracy: 0.4589
Epoch 6/50
7/7 [==============================] - 0s 22ms/step - loss: 1.2368 - accuracy: 0.4447 - val_loss: 1.2189 - val_accuracy: 0.4731
Epoch 7/50
7/7 [==============================] - 0s 20ms/step - loss: 1.2158 - accuracy: 0.4569 - val_loss: 1.2015 - val_accuracy: 0.5042
Epoch 8/50
7/7 [===========

Testing the model

In [108]:
y_test_pred = model9.predict(X_test)

In [110]:
y_test_pred = np.argmax(y_test_pred, axis=1)
y_test_pred 

array([3, 2, 0, 0, 3, 2, 3, 3, 2, 2, 3, 2, 3, 0, 2, 3, 1, 3, 0, 3, 0, 3,
       2, 2, 1, 0, 3, 0, 3, 3, 0, 1, 2, 2, 2, 3, 3, 1, 3, 2, 0, 3, 3, 1,
       2, 3, 2, 3, 2, 1, 1, 3, 1, 0, 2, 3, 3, 3, 0, 3, 0, 3, 1, 0, 2, 1,
       1, 3, 2, 3, 0, 3, 2, 3, 3, 1, 0, 3, 1, 2, 0, 3, 1, 3, 3, 2, 2, 3,
       0, 3, 3, 1, 1, 3, 0, 1, 0, 3, 0, 2, 2, 3, 0, 3, 3, 2, 3, 2, 1, 0,
       0, 3, 2, 2, 0, 2, 3, 2, 3, 3, 1, 2, 3, 3, 0, 0, 2, 3, 0, 0, 3, 2,
       3, 1, 0, 3, 3, 0, 1, 1, 1, 2, 3, 0, 3, 2, 3, 3, 3, 0, 0, 1, 3, 0,
       0, 0, 0, 2, 0, 2, 2, 1, 2, 1, 2, 3, 1, 3, 0, 1, 3, 1, 3, 2, 0, 1,
       3, 2, 0, 3, 0, 0, 2, 0, 0, 3, 0, 3, 3, 1, 1, 0, 3, 2, 3, 3, 3, 0,
       3, 3, 1, 2, 0, 0, 3, 1, 0, 0, 0, 3, 1, 3, 2, 2, 1, 3, 2, 2, 2, 3,
       0, 3, 2, 2, 2, 0, 2, 3, 2, 2, 3, 2, 0, 2, 2, 0, 0, 0, 3, 1, 1, 3,
       2, 3, 2, 1, 0, 0, 3, 1, 1, 3, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 2, 0,
       3, 0, 3, 0, 0, 3, 3, 2, 3, 1, 0, 1, 3, 1, 2, 2, 2, 3, 3, 1, 2, 2,
       3, 1, 2, 1, 3, 0, 3, 3, 3, 2, 1, 3, 2, 3, 0,

In [114]:
def normalize_value(predictions):
    normalized = []
    for prediction in predictions:
        result = 0
        for i in range(0,len(prediction)):
            result += i * prediction[i]
        normalized.append(int(result))
    return normalized

y_test_normalized = normalize_value(y_test)

In [115]:
print('Accuracy on test set : %.3f' % accuracy_score(y_test_normalized,y_test_pred))

Accuracy on test set : 0.712


Now, let us try hyperparameter tuning using automated tools

We can use GridSearchCV for this. We can also use Hyperas but GridSearch will do the job fine here too

In [141]:
def create_model():
  model = Sequential()

  model.add(Conv2D(filters=32, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(4, activation='softmax'))

  model.compile(optimizer=tf.keras.optimizers.Adam(0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
  
  return model

In [119]:
pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [122]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

Selecting the best batch size and no. of epochs

In [143]:
model = KerasClassifier(model=create_model, verbose=0)

In [144]:
batch_size = [16, 32, 64, 128]
epochs = [10, 25, 50]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train, validation_data=(X_val, y_val))

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [145]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.732630 using {'batch_size': 16, 'epochs': 50}


Selecting the best optimizer

In [146]:
def create_model2(optimizer='adam'):
  model = Sequential()

  model.add(Conv2D(filters=32, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(4, activation='softmax'))

  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  
  return model

In [148]:
model = KerasClassifier(model=create_model2, epochs=100, batch_size=10, verbose=0)

optimizer = ['SGD', 'RMSprop', 'Adam']
param_grid = dict(model__optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

In [149]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.822557 using {'model__optimizer': 'Adam'}


Selecting the best learning rate

In [151]:
def create_model3(optimizer='adam'):
  model = Sequential()

  model.add(Conv2D(filters=32, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(4, activation='softmax'))
  
  return model

In [152]:
model = KerasClassifier(model=create_model3, loss="categorical_crossentropy", optimizer="Adam", epochs=50, batch_size=16, verbose=0)

In [153]:
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(optimizer__learning_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

In [154]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.827419 using {'optimizer__learning_rate': 0.001}


Let us create a model with these hyperparameters and see

In [155]:
final = Sequential()

final.add(Conv2D(filters=32, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
final.add(Flatten())
final.add(Dense(64, activation='relu'))
final.add(Dense(32, activation='relu'))
final.add(Dense(4, activation='softmax'))

final.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
final.fit(X_train, y_train, batch_size=16, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
52/52 [==============================] - 1s 10ms/step - loss: 1.3850 - accuracy: 0.4095 - val_loss: 0.8950 - val_accuracy: 0.5581
Epoch 2/50
52/52 [==============================] - 0s 7ms/step - loss: 0.8465 - accuracy: 0.6051 - val_loss: 0.7366 - val_accuracy: 0.6431
Epoch 3/50
52/52 [==============================] - 0s 7ms/step - loss: 0.6314 - accuracy: 0.7606 - val_loss: 0.7020 - val_accuracy: 0.6912
Epoch 4/50
52/52 [==============================] - 0s 7ms/step - loss: 0.4553 - accuracy: 0.8408 - val_loss: 0.4459 - val_accuracy: 0.8329
Epoch 5/50
52/52 [==============================] - 0s 7ms/step - loss: 0.2449 - accuracy: 0.9320 - val_loss: 0.3779 - val_accuracy: 0.8555
Epoch 6/50
52/52 [==============================] - 0s 6ms/step - loss: 0.1364 - accuracy: 0.9733 - val_loss: 0.3608 - val_accuracy: 0.8697
Epoch 7/50
52/52 [==============================] - 0s 6ms/step - loss: 0.0665 - accuracy: 0.9915 - val_loss: 0.3543 - val_accuracy: 0.8669
Epoch 8/50
52/52 [=

In [156]:
y_test_pred = final.predict(X_test)
y_test_pred = np.argmax(y_test_pred, axis=1)

In [157]:
def normalize_value(predictions):
    normalized = []
    for prediction in predictions:
        result = 0
        for i in range(0,len(prediction)):
            result += i * prediction[i]
        normalized.append(int(result))
    return normalized

y_test_normalized = normalize_value(y_test)

In [158]:
print('Accuracy on test set : %.3f' % accuracy_score(y_test_normalized,y_test_pred))

Accuracy on test set : 0.875


We got a BIG boost in the test accuracy. This shows that we chose the best hyperparameters to train the model using GridSearch. Hence, we successfully tuned the hyperparameters of the CNN model